In [1]:
import sys
sys.path.append('..')

import os
os.environ['CUDA_VISIBLE_DEVICES'] = ''
import tensorflow as tf
import numpy as np
import pandas as pd
from model.model_fn import model_fn
from model.input_fn import input_fn
from model.utils import get_yaml_config

In [2]:
model_dir = '../experiments/search/swem_max_encoder=GRU_features=False/'
data_dir = '../data/full_stratified/'

params = get_yaml_config(os.path.join(model_dir, 'config.yaml'))

params['treatments_vocab_path'] = os.path.join(data_dir, 'treatments.txt')

../model/utils.py:14: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  params = yaml.load(f)


In [3]:
config = tf.estimator.RunConfig(model_dir=model_dir)
estimator = tf.estimator.Estimator(model_fn, config=config, params=params)

INFO:tensorflow:Using config: {'_model_dir': '../experiments/search/swem_max_encoder=GRU_features=False/', '_tf_random_seed': None, '_save_summary_steps': 100, '_save_checkpoints_steps': None, '_save_checkpoints_secs': 600, '_session_config': allow_soft_placement: true
graph_options {
  rewrite_options {
    meta_optimizer_iterations: ONE
  }
}
, '_keep_checkpoint_max': 5, '_keep_checkpoint_every_n_hours': 10000, '_log_step_count_steps': 100, '_train_distribute': None, '_device_fn': None, '_protocol': None, '_eval_distribute': None, '_experimental_distribute': None, '_service': None, '_cluster_spec': <tensorflow.python.training.server_lib.ClusterSpec object at 0x7fcc7282a630>, '_task_type': 'worker', '_task_id': 0, '_global_id_in_cluster': 0, '_master': '', '_evaluation_master': '', '_is_chief': True, '_num_ps_replicas': 0, '_num_worker_replicas': 1}


In [4]:
train_path = os.path.join(data_dir, 'train.csv')
eval_path = os.path.join(data_dir, 'eval.csv')

train_input = lambda: input_fn(train_path, params, False)
eval_input = lambda: input_fn(eval_path, params, False)

train_preds = estimator.predict(train_input)
eval_preds = estimator.predict(eval_input)

In [5]:
train_hidden = []
for p in train_preds:
    train_hidden.append(p['hidden'])

eval_hidden = []
for p in eval_preds:
    eval_hidden.append(p['hidden'])
    
train_hidden = np.array(train_hidden)
eval_hidden = np.array(eval_hidden)

Instructions for updating:
Colocations handled automatically by placer.

For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Calling model_fn.
Instructions for updating:
This class is equivalent as tf.keras.layers.GRUCell, and will be replaced by that in Tensorflow 2.0.
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
Use keras.layers.dense instead.
INFO:tensorflow:Done calling model_fn.
INFO:tensorflow:Graph was finalized.
Instructions for updating:
Use standard file APIs to check for files with this prefix.
INFO:tensorflow:Restoring parameters from ../experiments/search/swem_max_encoder=GRU_features=False/model.ckpt-390
INFO:tensorflow:Running local_init_op.
INFO:tensorflow:Done running local_init_op.
INFO:tensorflo

In [8]:
train_labels = pd.read_csv(train_path)['target'].values
eval_labels = pd.read_csv(eval_path)['target'].values

In [13]:
train_hidden.shape, train_labels.shape

((266709, 128), (266709,))

In [14]:
np.save('train_hidden.npy', train_hidden)
np.save('train_labels.npy', train_labels)

In [15]:
eval_hidden.shape, eval_labels.shape

((114304, 128), (114304,))

In [16]:
np.save('eval_hidden.npy', eval_hidden)
np.save('eval_labels.npy', eval_labels)